## Using last 7th attention layer with window size of 7

We extract the attention distribution directly from a model that has been pretrained on bias detection! We then using windowing to extract a window of attention scores around the biased word.

In [17]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist, return_idx_attention_dist, window_attention_dist
from src.utils.classification_utils import run_bootstrapping
from src.utils.shared_utils import get_bias_predictions

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
params = Params.read_params("linear-params-window.json")
print("model = {}".format(params.final_task['model']))
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

model = shallow_nn
layers = [7]
reducer = avg


In [4]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)

03/26/2020 19:02:50 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 3653.42it/s]


### Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [5]:
attention_dataloader = dataset.return_dataloader(batch_size=params.intermediary_task['attention']['attention_extraction_batch_size']) 
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True)

03/26/2020 19:02:52 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/26/2020 19:02:52 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/26/2020 19:02:52 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmppyr7xstm
03/26/2020 19:02:57 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

Instantiated joint model with pretrained weights.
Succesfully loaded in attention experiment!


```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [6]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

Getting the predictions from the BERT model trained to detect bias, and using those to index into the attention scores

In [7]:
bias_predictions = get_bias_predictions(dataset, params.intermediary_task, params.dataset, batch_size=8)

03/26/2020 19:03:22 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/26/2020 19:03:22 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/26/2020 19:03:22 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpyhy3fwhw
03/26/2020 19:03:26 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

In [8]:
bias_indices = torch.argmax(bias_predictions == 1, dim=1).tolist()

In [18]:
attention_scores_indexed = return_idx_attention_dist(attention_scores, bias_indices)
reduced_attention = reduce_attention_dist(attention_scores_indexed, params.intermediary_task["attention"]["reducer"])
reduced_attention = torch.cat(reduced_attention)
windowed_dist = window_attention_dist(reduced_attention, bias_indices, window_size=7)
attention_dist = windowed_dist

In [21]:
dataset.add_data(attention_dist, "attention_dist")
dataset.shuffle_data()
assert(attention_dist.shape[1] == params.final_task['input_dim'])

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [20]:
params = Params.read_params("linear-params-window.json")

In [22]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [23]:
stats = run_bootstrapping(classification_experiment, dataset, params.final_task, num_bootstrap_iters=10, input_key="attention_dist", label_key="bias_label")

In [24]:
params.final_task

{'model': 'shallow_nn',
 'dropout_prob': 0,
 'input_dim': 15,
 'hidden_dim': 4,
 'output_dim': 1,
 'data_split': {'train_split': 0.8, 'eval_split': 0.2, 'test_split': 0},
 'training_params': {'optimizer': 'adam',
  'loss': 'bce_with_logits',
  'num_epochs': 120,
  'batch_size': 32,
  'lr': 0.001}}

In [25]:
stats

{'auc': [(0.4335020242914979, 0.6072368421052631), 0.520242914979757],
 'accuracy': [(0.44843750000000004, 0.59375), 0.575]}

In [26]:
classification_experiment.save_model_weights("linear-windowed-attention.weights")